In [ ]:
%run base.ipynb

In [ ]:
experiment = 'all-mpnet-baseline'
experiment = 'all-mpnet-mulneg'
experiment = 'all-mpnet-mulneg-beginning'


results_path = get_results_path(experiment)

In [ ]:
from kialo_util import read_maps
maps = read_maps(results_path / 'all/all_maps_embedding.pkl')
samples = read_maps(results_path / 'annotated_samples_embeddings.pkl')
# maps = read_maps(Path('/home/ij/Downloads/Code/deliberatorium/results/results/annotated_samples_embeddings.pkl'))

## Distance between parent/child based on level 

In [ ]:
from numpy import dot
assert 'beginning' in experiment
maps_embeddings = [[{'child': x.embedding, 'parent': x.extra_embeddings['parent'], 
                     'level': x.get_level(), 
                     'cosine': dot(x.embedding, x.extra_embeddings['parent'])} for x in m.all_children] for m in maps]
levels, cosine = [x['level'] for m in maps_embeddings for x in m], [x['cosine'] for m in maps_embeddings for x in m]

In [ ]:
t = 7
import seaborn as sns
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8, 6)) 
sns.lineplot([x if x<t else t for x in levels], cosine, )
plt.xlabel('level', fontsize=16)
plt.ylabel('cosine similarity',fontsize=18)
fig.savefig(f'output/{experiment}-same-node-cosine.pdf', format='pdf')

In [ ]:
import pandas as pd
df = pd.DataFrame({'level': levels, 'cosine': cosine})
df.groupby('level').agg(['mean', 'count'])

In [ ]:
for x in range(8):
    print(x, df[df['level']==x]['cosine'].mean(), df[df['level']>x]['cosine'].mean())

## Distance between c -> p vs p -> c

In [ ]:
import statistics
actual_parent_cosine = [dot(x.embedding, x.parent.extra_embeddings['parent'] if 'beginning' in experiment else x.parent.embedding) for m in maps for x in m.all_children if x.parent]
inverse_parent_cosine = [dot(x.extra_embeddings['parent'], x.parent.embedding if 'beginning' in experiment else x.parent.embedding) for m in maps for x in m.all_children if x.parent]
len(actual_parent_cosine), statistics.mean(actual_parent_cosine), statistics.mean(inverse_parent_cosine)

## Distance between all nodes in a map

In [ ]:
import statistics
import numpy as np
embeddings_similarity_all, extra_embeddings_similarity_all, mixed_embeddings_similarity_all= [], [], []
for m in maps:
    embeddings = [x.embedding for x in m.all_children]
    embeddings_similarity = dot(embeddings, np.transpose(embeddings))
    embeddings_similarity_all.append(embeddings_similarity.mean())
    if 'beginning' in experiment:
        extra_embeddings = [x.extra_embeddings['parent'] for m in maps for x in m.all_children]
        extra_embeddings_similarity = dot(extra_embeddings, np.transpose(extra_embeddings))
        extra_embeddings_similarity_all.append(extra_embeddings_similarity.mean())
        mixed_embeddings_similarity = dot(embeddings, np.transpose(extra_embeddings))
        mixed_embeddings_similarity_all.append(mixed_embeddings_similarity.mean())
        
print(experiment, statistics.mean(embeddings_similarity_all))
if 'beginning' in experiment:
    print(statistics.mean(mixed_embeddings_similarity_all))
    print(statistics.mean(extra_embeddings_similarity_all))

In [ ]:
[len(x) for x in [embeddings_similarity_all, extra_embeddings_similarity_all, mixed_embeddings_similarity_all]]

In [ ]:
len(maps)

## Distance between all nodes in all maps

In [ ]:
import numpy as np
embeddings = [x.embedding for m in maps for x in m.all_children]
embeddings_similarity = dot(embeddings, np.transpose(embeddings))
print(experiment, embeddings_similarity.mean())
if 'beginning' in experiment:
    extra_embeddings = [x.extra_embeddings['parent'] for m in maps for x in m.all_children]
    extra_embeddings_similarity = dot(embeddings, np.transpose(extra_embeddings))
    print(extra_embeddings_similarity.mean())
    extra_embeddings_similarity = dot(extra_embeddings, np.transpose(extra_embeddings))
    print(extra_embeddings_similarity.mean())

## Visualize Embeddings 

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

def tsne_plot(embeddings, labels, weights, output=None, extra_embeddings=None, extra_weights=None,):    
#     tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=500, random_state=42)
#     tsne_model = TSNE(perplexity=250*(2 if extra_embeddings else 1), n_components=2, init='pca', n_iter=1000, random_state=42)
    tsne_model = TSNE(perplexity=500, n_components=2, init='pca', n_iter=1000, random_state=42)
    emb2d = tsne_model.fit_transform(embeddings if not extra_embeddings else embeddings+extra_embeddings)
        
    fig = plt.figure(figsize=(10, 10)) 
    x, y = emb2d[:,0],emb2d[:,1]
    if not extra_embeddings:
        g = plt.scatter(x, y, c=weights, cmap='Blues', vmin=0, alpha=0.5)
    else:
        plt.scatter(x[len(embeddings):], y[len(embeddings):], c=extra_weights, cmap='Oranges', vmin=0, alpha=0.5)
        plt.scatter(x[:len(embeddings)], y[:len(embeddings)], c=weights, cmap='Blues', vmin=0, alpha=0.5)
#         plt.colorbar()
#         plt.show()
#         plt.figure(figsize=(10, 10)) 
    
    if len(labels) < 100:
        for i in range(len(x)):
            plt.annotate(labels[i][:100],
                         xy=(x[i], y[i]),
                         xytext=(5, 2),
                         textcoords='offset points',
                         ha='right',
                         va='bottom')
            
#     plt.colorbar()

    fig.savefig(f'output/{experiment}-{output if output else ""}.pdf', format='pdf')
    
    plt.show()

In [ ]:
sample = list(samples.values())[0]
tsne_plot(np.vstack([sample['candidates_embedding'], sample['embedding']]), 
          [x['text'] for x in list(sample['candidates'].values())]+[sample['text']], ([1]*len(sample['candidates'])+[5]), output='samples')
print(sample['text'])
print(sample['candidates'][sample['parent ID']]['text'])

In [ ]:
# np.array([x.embedding for x in maps[0].all_children]).shape
# tsne_plot([x.embedding for x in maps[0].all_children], [x.get_level() for x in maps[0].all_children])
if not 'beginning' in experiment:
    tsne_plot([x.embedding for x in maps[0].all_children], [x.name for x in maps[0].all_children], [10-x.get_level() for x in maps[0].all_children], output='map')
else:
    tsne_plot([x.embedding for x in maps[0].all_children], 
              [x.name for x in maps[0].all_children], 
              [10-x.get_level() for x in maps[0].all_children],
              extra_embeddings=[x.extra_embeddings['parent'] for x in maps[0].all_children],
              extra_weights=[10-x.get_level() for x in maps[0].all_children],
              output='map'
         )